# 2nd Level - Fixed Effects (FFX)

In [1]:
from nipype import Node, Workflow, IdentityInterface, Function, MapNode,SelectFiles, DataSink
from nipype.interfaces import fsl
from nipype.algorithms.modelgen import SpecifyModel

import numpy as np
import os

In [2]:
def sort_inputs(copes,varcopes,num_of_runs):
    """
    This function sorts the copes and varcopes for each subject
    """
    copes.sort(key = lambda x: x.split("/")[-1])
    cope_list =  [copes[i:i + num_of_runs] for i in range(0, len(copes), num_of_runs)] 
    varcopes.sort(key = lambda x: x.split("/")[-1])
    varcope_list =  [varcopes[i:i + num_of_runs] for i in range(0, len(varcopes), num_of_runs)] 
    return cope_list,varcope_list

In [3]:
# path initialization 
data = r"/home/anakin/Desktop/ds000113/output/1st_level"
output = r"/home/anakin/Desktop/ds000113/output/2nd_level"
mask_file = "/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz"

# again - 8 subjects and 2 runs to be averaged for each sub
num_of_subjects = 10 
num_of_runs = 2
excluded = [7,8]
subject_list = ["{:02d}".format(i) for i in range(1,num_of_subjects+1) if i not in excluded]

# put in number of contrasts based on 1st level
num_of_cont = 15

In [9]:
# nodes
infosource = Node(IdentityInterface(fields=["subject_list"]), name="infosource")
infosource.iterables = [('subject_list', subject_list)]

templates = {"copes":     "modelestimate/{subject_list}/modelestimate_run_*/cope*.nii.gz",
             "varcopes":  "modelestimate/{subject_list}/modelestimate_run_*/varcope*.nii.gz",
             "dof_files" : "modelestimate/{subject_list}/modelestimate_run_*/dof"}

selectfiles = Node(SelectFiles(templates),name="selectfiles")
selectfiles.inputs.base_directory = data
substitutions = []
substitutions += [("_subject_list_{:02d}".format(i), "{:02d}".format(i)) for i in range(1,num_of_subjects+1) if i not in excluded]
substitutions += [("_copemerge", "copemerge")]
substitutions += [("_varcopemerge", "varcopemerge")]

datasink = Node(DataSink(base_directory=output),name="datasink")
datasink.inputs.substitutions = substitutions
datasink.output = output

sort_copes_varcopes = Node(Function(input_names=["copes","varcopes","num_of_runs"],
                                  output_names=["cope_list","varcope_list"],
                                  function=sort_inputs),
                                  name='sort_copes_varcopes')

sort_copes_varcopes.inputs.num_of_runs = num_of_runs

copemerge = MapNode(fsl.Merge(),iterfield=['in_files'],name="copemerge")
copemerge.inputs.dimension = "t"

varcopemerge = MapNode(fsl.Merge(),iterfield=['in_files'],name="varcopemerge")
varcopemerge.inputs.dimension = "t"

level2model = Node(interface=fsl.L2Model(), name='l2model')
level2model.inputs.num_copes = num_of_runs

fixed_eff_model = MapNode(interface=fsl.FLAMEO(),name="fixed_eff_model",iterfield=['cope_file', 'var_cope_file'])
fixed_eff_model.inputs.run_mode='fe'
fixed_eff_model.inputs.mask_file= mask_file

In [10]:
# workflow creation and  nodeconnection
fixed_effects_analysis = Workflow(name="fixed_effects_analysis")

fixed_effects_analysis.connect(infosource,"subject_list",datasink,"container")
fixed_effects_analysis.connect(infosource,"subject_list",selectfiles,"subject_list")
fixed_effects_analysis.connect(selectfiles,"copes",sort_copes_varcopes,"copes")
fixed_effects_analysis.connect(selectfiles,"varcopes",sort_copes_varcopes,"varcopes")
fixed_effects_analysis.connect(sort_copes_varcopes,"cope_list",copemerge,"in_files")
fixed_effects_analysis.connect(sort_copes_varcopes,"varcope_list",varcopemerge,"in_files")
fixed_effects_analysis.connect(copemerge,"merged_file",fixed_eff_model,"cope_file")
fixed_effects_analysis.connect(varcopemerge,"merged_file",fixed_eff_model,"var_cope_file")   
fixed_effects_analysis.connect(level2model,"design_mat",fixed_eff_model,"design_file")   
fixed_effects_analysis.connect(level2model,"design_con",fixed_eff_model,"t_con_file") 
fixed_effects_analysis.connect(level2model,"design_grp",fixed_eff_model,"cov_split_file") 

In [11]:
fixed_effects_analysis.write_graph(graph2use='flat', format='png', simple_form=True,
                          dotfilename= os.path.join(output,"fixed_effects_analysis_workflow.dot"))

# Visualize workflow without datasink nodes
img = Image.open(os.path.join(output,"fixed_effects_analysis_workflow.png"))
img.show()

231012-23:20:35,639 nipype.workflow INFO:
	 Generated workflow graph: /home/anakin/Desktop/ds000113/output/2nd_level/fixed_effects_analysis_workflow.png (graph2use=flat, simple_form=True).


In [12]:
# connect datasink
fixed_effects_analysis.connect(selectfiles,"copes",datasink,"copes")
fixed_effects_analysis.connect(selectfiles,"varcopes",datasink,"varcopes")
fixed_effects_analysis.connect(selectfiles,"dof_files",datasink,"dof_files")
fixed_effects_analysis.connect(copemerge,"merged_file",datasink,"merge.@merged_cope_file")
fixed_effects_analysis.connect(varcopemerge,"merged_file",datasink,"merge.@merged_varcope_file")
fixed_effects_analysis.connect(level2model,"design_mat",datasink,"model.@design_mat")
fixed_effects_analysis.connect(fixed_eff_model,"copes",datasink,"fixed_effects.@copes")
fixed_effects_analysis.connect(fixed_eff_model,"var_copes",datasink,"fixed_effects.@var_copes")
fixed_effects_analysis.connect(fixed_eff_model,"weights",datasink,"fixed_effects.@weights")
fixed_effects_analysis.connect(fixed_eff_model,"zstats",datasink,"fixed_effects.@zstats")
fixed_effects_analysis.connect(fixed_eff_model,"res4d",datasink,"fixed_effects.@res4d")
fixed_effects_analysis.connect(fixed_eff_model,"tstats",datasink,"fixed_effects.@tstats")

In [13]:
# run second level (FFX)
fixed_effects_analysis.run(plugin='MultiProc', plugin_args={'n_procs' : 1})

231012-23:20:35,792 nipype.workflow INFO:
	 Workflow fixed_effects_analysis settings: ['check', 'execution', 'logging', 'monitoring']
231012-23:20:35,877 nipype.workflow INFO:
	 Running in parallel.
231012-23:20:36,20 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 9 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-23:20:36,89 nipype.workflow INFO:
	 [Node] Setting-up "fixed_effects_analysis.l2model" in "/tmp/tmpnm8c3lj2/fixed_effects_analysis/l2model".
231012-23:20:36,95 nipype.workflow INFO:
	 [Node] Executing "l2model" <nipype.interfaces.fsl.model.L2Model>
231012-23:20:36,99 nipype.workflow INFO:
	 [Node] Finished "l2model", elapsed time 0.000672s.
231012-23:20:38,28 nipype.workflow INFO:
	 [Job 0] Completed (fixed_effects_analysis.l2model).
231012-23:20:38,40 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-23:20:38,116 nipype.workflow INFO:
	 [Node] Setting-up "fixed_e

231012-23:21:00,105 nipype.workflow INFO:
	 [Node] Finished "sort_copes_varcopes", elapsed time 0.001s.
231012-23:21:02,51 nipype.workflow INFO:
	 [Job 12] Completed (fixed_effects_analysis.sort_copes_varcopes).
231012-23:21:02,59 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 96 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-23:21:02,101 nipype.workflow INFO:
	 [Node] Setting-up "fixed_effects_analysis.sort_copes_varcopes" in "/tmp/tmpz4i5stoq/fixed_effects_analysis/_subject_list_05/sort_copes_varcopes".
231012-23:21:02,114 nipype.workflow INFO:
	 [Node] Executing "sort_copes_varcopes" <nipype.interfaces.utility.wrappers.Function>
231012-23:21:02,117 nipype.workflow INFO:
	 [Node] Finished "sort_copes_varcopes", elapsed time 0.000576s.
231012-23:21:04,54 nipype.workflow INFO:
	 [Job 13] Completed (fixed_effects_analysis.sort_copes_varcopes).
231012-23:21:04,61 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 125 jobs ready. Free memory (GB): 

231012-23:21:28,82 nipype.workflow INFO:
	 [Job 57] Completed (_copemerge8).
231012-23:21:28,89 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 231 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-23:21:28,138 nipype.workflow INFO:
	 [Node] Setting-up "_copemerge9" in "/tmp/tmpor_6x2na/fixed_effects_analysis/_subject_list_01/copemerge/mapflow/_copemerge9".
231012-23:21:28,142 nipype.workflow INFO:
	 [Node] Executing "_copemerge9" <nipype.interfaces.fsl.utils.Merge>
231012-23:21:28,304 nipype.workflow INFO:
	 [Node] Finished "_copemerge9", elapsed time 0.159564s.
231012-23:21:30,84 nipype.workflow INFO:
	 [Job 58] Completed (_copemerge9).
231012-23:21:30,87 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 230 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-23:21:30,133 nipype.workflow INFO:
	 [Node] Setting-up "_copemerge10" in "/tmp/tmpor_6x2na/fixed_effects_analysis/_subject_list_01/copemerge/mapflow/_copemerge10".
231012

231012-23:21:40,279 nipype.workflow INFO:
	 [Node] Cached "_copemerge12" - collecting precomputed outputs
231012-23:21:40,280 nipype.workflow INFO:
	 [Node] "_copemerge12" found cached.
231012-23:21:40,283 nipype.workflow INFO:
	 [Node] Setting-up "_copemerge13" in "/tmp/tmpor_6x2na/fixed_effects_analysis/_subject_list_01/copemerge/mapflow/_copemerge13".
231012-23:21:40,287 nipype.workflow INFO:
	 [Node] Cached "_copemerge13" - collecting precomputed outputs
231012-23:21:40,292 nipype.workflow INFO:
	 [Node] "_copemerge13" found cached.
231012-23:21:40,295 nipype.workflow INFO:
	 [Node] Setting-up "_copemerge14" in "/tmp/tmpor_6x2na/fixed_effects_analysis/_subject_list_01/copemerge/mapflow/_copemerge14".
231012-23:21:40,298 nipype.workflow INFO:
	 [Node] Cached "_copemerge14" - collecting precomputed outputs
231012-23:21:40,299 nipype.workflow INFO:
	 [Node] "_copemerge14" found cached.
231012-23:21:42,98 nipype.workflow INFO:
	 [Job 17] Completed (fixed_effects_analysis.copemerge).
23

231012-23:22:04,371 nipype.workflow INFO:
	 [Node] Finished "_varcopemerge11", elapsed time 0.167183s.
231012-23:22:06,124 nipype.workflow INFO:
	 [Job 75] Completed (_varcopemerge11).
231012-23:22:06,128 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 213 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-23:22:06,187 nipype.workflow INFO:
	 [Node] Setting-up "_varcopemerge12" in "/tmp/tmpr_5c01af/fixed_effects_analysis/_subject_list_01/varcopemerge/mapflow/_varcopemerge12".
231012-23:22:06,191 nipype.workflow INFO:
	 [Node] Executing "_varcopemerge12" <nipype.interfaces.fsl.utils.Merge>
231012-23:22:06,360 nipype.workflow INFO:
	 [Node] Finished "_varcopemerge12", elapsed time 0.165708s.
231012-23:22:08,128 nipype.workflow INFO:
	 [Job 76] Completed (_varcopemerge12).
231012-23:22:08,131 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 212 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-23:22:08,176 nipype.workflow INFO:
	

...

231012-23:38:05,208 nipype.workflow INFO:
	 [Node] Cached "_fixed_eff_model0" - collecting precomputed outputs
231012-23:38:05,209 nipype.workflow INFO:
	 [Node] "_fixed_eff_model0" found cached.
231012-23:38:05,212 nipype.workflow INFO:
	 [Node] Setting-up "_fixed_eff_model1" in "/tmp/tmp4tp0rj8x/fixed_effects_analysis/_subject_list_10/fixed_eff_model/mapflow/_fixed_eff_model1".
231012-23:38:05,215 nipype.workflow INFO:
	 [Node] Cached "_fixed_eff_model1" - collecting precomputed outputs
231012-23:38:05,216 nipype.workflow INFO:
	 [Node] "_fixed_eff_model1" found cached.
231012-23:38:05,219 nipype.workflow INFO:
	 [Node] Setting-up "_fixed_eff_model2" in "/tmp/tmp4tp0rj8x/fixed_effects_analysis/_subject_list_10/fixed_eff_model/mapflow/_fixed_eff_model2".
231012-23:38:05,223 nipype.workflow INFO:
	 [Node] Cached "_fixed_eff_model2" - collecting precomputed outputs
231012-23:38:05,224 nipype.workflow INFO:
	 [Node] "_fixed_eff_model2" found cached.
231012-23:38:05,229 nipype.workflow IN

231012-23:38:07,347 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/merge/_subject_list_10/_copemerge9/cope10_merged.nii.gz -> /home/anakin/Desktop/ds000113/output/2nd_level/10/merge/10/copemerge9/cope10_merged.nii.gz
231012-23:38:07,349 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/merge/_subject_list_10/_copemerge10/cope11_merged.nii.gz -> /home/anakin/Desktop/ds000113/output/2nd_level/10/merge/10/copemerge10/cope11_merged.nii.gz
231012-23:38:07,351 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/merge/_subject_list_10/_copemerge11/cope12_merged.nii.gz -> /home/anakin/Desktop/ds000113/output/2nd_level/10/merge/10/copemerge11/cope12_merged.nii.gz
231012-23:38:07,357 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/merge/_subject_list_10/_copemerge12/cope13_merged.nii.gz -> /home/anakin/Desktop/ds000113/output/2nd_level/10/merge/10/copemerge12/cope13_merged.n

231012-23:38:07,409 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/_subject_list_10/_fixed_eff_model11/cope1.nii.gz -> /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/10/_fixed_eff_model11/cope1.nii.gz
231012-23:38:07,414 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/_subject_list_10/_fixed_eff_model12/cope1.nii.gz -> /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/10/_fixed_eff_model12/cope1.nii.gz
231012-23:38:07,416 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/_subject_list_10/_fixed_eff_model13/cope1.nii.gz -> /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/10/_fixed_eff_model13/cope1.nii.gz
231012-23:38:07,418 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/_subject_list_10/_fixed_eff_model14/cope1.nii.gz -> /home/anakin/Desktop/ds000113/output/2n

231012-23:38:07,503 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/_subject_list_10/_fixed_eff_model12/weights1.nii.gz -> /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/10/_fixed_eff_model12/weights1.nii.gz
231012-23:38:07,505 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/_subject_list_10/_fixed_eff_model13/weights1.nii.gz -> /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/10/_fixed_eff_model13/weights1.nii.gz
231012-23:38:07,506 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/_subject_list_10/_fixed_eff_model14/weights1.nii.gz -> /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/10/_fixed_eff_model14/weights1.nii.gz
231012-23:38:07,508 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/_subject_list_10/_fixed_eff_model0/zstat1.nii.gz -> /home/anakin/Desktop/

231012-23:38:07,564 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/_subject_list_10/_fixed_eff_model13/res4d.nii.gz -> /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/10/_fixed_eff_model13/res4d.nii.gz
231012-23:38:07,566 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/_subject_list_10/_fixed_eff_model14/res4d.nii.gz -> /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/10/_fixed_eff_model14/res4d.nii.gz
231012-23:38:07,572 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/_subject_list_10/_fixed_eff_model0/tstat1.nii.gz -> /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/10/_fixed_eff_model0/tstat1.nii.gz
231012-23:38:07,574 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/2nd_level/10/fixed_effects/_subject_list_10/_fixed_eff_model1/tstat1.nii.gz -> /home/anakin/Desktop/ds000113/output/2n

In [14]:
print('second level is done!')

second level is done!


clean up tmp folder using the below command in command window:

sudo rm -r /tmp/*